In [1]:
# pip install duckdb
# pip install jupysql
# pip install notebook
# pip install tabulate
# pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [3]:
# If on windows, this only works if you're running Jupyter as an admin
# pip install duckdb-engine

  Using cached duckdb_engine-0.13.1-py3-none-any.whl (47 kB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
import sqlalchemy
from datetime import date
import json
import pathlib
%load_ext sql

Deploy Panel apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [2]:
# Set up SQL connection
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Change the billboard_observation_date!

In [3]:
current_date = date.today().strftime('%Y-%m-%d')
billboard_observation_date = '2024-07-31'

In [4]:
# Replace with your filepath. Forward slashes work for both Mac and Windows
%sql duckdb:///C:/Users/Admin/billboard/raw_db.duckdb

In [7]:
# Test connection
%sql USE raw_db;
%sql USE raw_db.billboard;
%sql SELECT * FROM raw_db.billboard.artist LIMIT 2;

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
1,2,Morgan Wallen,0.0,2024-08-23


In [8]:
# Update this for this week's rankings
# Easier in an IDE that allows opt+arrow_key to move lines up and down
# Update this for this week's rankings
rankings = [
    'A Bar Song (Tipsy)'
    , 'I Had Some Help'
    , 'Lies Lies Lies'    
    , "Ain't No Love In Oklahoma"
    , 'Cowgirls'
    , 'Pink Skies'
    , '28'
    , 'Miles On It'
    , 'High Road'
    , 'Austin'
    , 'I Am Not Okay'
    , 'Pour Me A Drink'
    , 'Whiskey Whiskey'
    , 'Bulletproof'
    , 'Dirt Cheap'
    , 'You Look Like You Love Me'
    , 'American Nights'
    , "Wind Up Missin' You"
    , 'Sweet Dreams'
    , 'Chevrolet'
    , "Think I'm In Love With You"
    , 'Beautiful As You'
    , 'Hang Tight Honey'
    , 'Cowboys Cry Too'
    , 'Better Days'
    , 'Indifferent'
    , 'Bass Boat'
    , 'Oak Island'
    , 'Sandpaper'
    , 'Dead End Road'
    , 'Love You, Miss You, Mean It'
    , 'Love You Again'
    , 'Am I Okay?'
    , "She's Somebody's Daughter (Reimagined)"
    , 'Holy Smokes'
    , 'New To Country'
    , 'Devil You Know'
    , 'Worst Way'
    , 'The Way Back'
    , 'Young Love & Saturday Nights'
    , "What He'll Never Have"
    , 'Nine Ball'
    , 'Remember Him That Way'
    , 'Maxed Out'
    , 'Out Of Oklahoma'
    , 'The Great American Bar Scene'
    , 'Boys Back Home'
    , 'Mama I Lied'
    , 'Relapse'
    , 'Purple Gas'
]

In [9]:
%%sql df_song_labeled <<
SELECT
    ss.song_id
    , ss.song_name
    , art.artist_name
    , s2a.relationship_type
    , ss.untracked_weeks_on_chart
FROM song ss
LEFT JOIN song_to_artist s2a
    ON ss.song_id = s2a.song_id
LEFT JOIN artist art
    ON s2a.artist_id = art.artist_id
;

In [10]:
df_song_labeled.tail(20)

,song_id,song_name,artist_name,relationship_type,untracked_weeks_on_chart
102,17,Blackbiird,Tanner Adell,Other main artist,0.0
103,17,Blackbiird,Beyonce,Lead artist,0.0
104,50,Whiskey Whiskey,Moneybagg Yo,Lead artist,0.0
105,2,Last Night,None,None,51.0
106,3,Thinkin' Bout Me,None,None,46.0
107,4,Fast Car,None,None,43.0
108,5,White Horse,None,None,26.0
109,6,World On Fire,None,None,32.0
110,7,Need A Favor,None,None,51.0
111,8,Pretty Little Poison,None,None,32.0


In [11]:
existing_song_to_rank = {}
new_song_to_rank = {}
for song, rank_ix in zip(rankings, range(1,51)):
    print(df_song_labeled.loc[df_song_labeled['song_name'] == song].to_markdown())
    song_id = input("Please enter the song_id or 'New Song'\n")

    if song_id == '': # hit enter to take the most common song ID
        song_id = int(
            df_song_labeled.loc[
                df_song_labeled['song_name'] == song
            ]['song_id'].mode()
        )
        
    if str(song_id).lower() == 'new song':
        new_song_to_rank[song] = rank_ix
    else:
        existing_song_to_rank[int(song_id)] = rank_ix

|    |   song_id | song_name          | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-------------------|:--------------|:--------------------|---------------------------:|
| 16 |        34 | A Bar Song (Tipsy) | Shaboozey     | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 20 |        38 | I Had Some Help | Morgan Wallen | Other main artist   |                          0 |
| 76 |        38 | I Had Some Help | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name      | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:---------------|:--------------|:--------------------|-------

|    |   song_id | song_name        | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-----------------|:--------------|:--------------------|---------------------------:|
| 21 |        39 | Beautiful As You | Thomas Rhett  | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name        | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-----------------|:--------------|:--------------------|---------------------------:|
| 24 |        42 | Hang Tight Honey | Lainey Wilson | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

|    |   song_id | song_name       | artist_name      | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:-----------------|:--------------------|---------------------------:|
| 35 |        55 | Cowboys Cry Too | Noah Kahan       | Other main artist   

Please enter the song_id or 'New Song'

|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 42 |        63 | Out Of Oklahoma | Lainey Wilson | Lead artist         |                          0 |
Please enter the song_id or 'New Song'

| song_id   | song_name   | artist_name   | relationship_type   | untracked_weeks_on_chart   |
|-----------|-------------|---------------|---------------------|----------------------------|
Please enter the song_id or 'New Song'
New Song
| song_id   | song_name   | artist_name   | relationship_type   | untracked_weeks_on_chart   |
|-----------|-------------|---------------|---------------------|----------------------------|
Please enter the song_id or 'New Song'
New Song
|    |   song_id | song_name   | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:------------|

In [12]:
print(json.dumps(existing_song_to_rank, indent = 2))
print(json.dumps(new_song_to_rank, indent = 2))

{
  "34": 1,
  "38": 2,
  "58": 3,
  "40": 4,
  "32": 5,
  "43": 6,
  "57": 7,
  "37": 8,
  "47": 9,
  "27": 10,
  "48": 11,
  "52": 12,
  "50": 13,
  "24": 14,
  "21": 15,
  "53": 16,
  "56": 17,
  "30": 18,
  "41": 19,
  "25": 20,
  "22": 21,
  "39": 22,
  "42": 23,
  "55": 24,
  "59": 25,
  "72": 26,
  "61": 27,
  "60": 28,
  "71": 29,
  "33": 31,
  "23": 32,
  "62": 33,
  "44": 34,
  "26": 35,
  "75": 36,
  "36": 37,
  "35": 38,
  "73": 39,
  "29": 40,
  "31": 41,
  "28": 42,
  "51": 43,
  "46": 44,
  "63": 45,
  "74": 48,
  "49": 49
}
{
  "Dead End Road": 30,
  "The Great American Bar Scene": 46,
  "Boys Back Home": 47,
  "Purple Gas": 50
}


## Add artists on any new songs that aren't in the artist table yet to its CSV

- Dead End Road: Jelly Roll
- The Great American Bar Scene: Zach Bryan
- Boys Back Home: Dylan Marlowe & Dylan Scott (+6)
- Purple Gas: Zach Bryan & Noeline Hofmann

In [13]:
df_artist = pd.read_csv('source_data/artist.csv')

In [14]:
new_song_artists = [
    'Jelly Roll' # 5
    , 'Zach Bryan' # 1
    , 'Dylan Marlowe' # new
    , 'Dylan Scott' # 7
    , 'Noeline Hofmann' # new
]
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
4,5,Jelly Roll,0.0,2024-08-23
6,7,Dylan Scott,0.0,2024-08-23


In [15]:
df_artist['artist_id'].max()

61

In [16]:
df_artist = pd.concat(
    [
        df_artist
        , pd.DataFrame(
            np.array([
                [62, 'Dylan Marlowe', 0, current_date]
                , [63, 'Noeline Hofmann', 1, current_date]
            ])
            , columns = ['artist_id', 'artist_name', 'pct_female', 'last_update']
        )
    ]
    , ignore_index = True
)

In [17]:
df_artist.to_csv('source_data/artist.csv', index=False)

## Add all new songs to the song table CSV

In [18]:
df_song_raw = pd.read_csv('source_data/song.csv')

In [20]:
max_song_id = df_song_raw['song_id'].max()
array_new_songs = []
for song_name in [
    'Dead End Road' # 5
    , 'The Great American Bar Scene' # 1
    , 'Boys Back Home' # 62 & 7 (+6)
    , 'Purple Gas' # 1 & 63
]:
    max_song_id += 1
    array_new_songs += [[max_song_id, song_name, 0, current_date]]

array_new_songs

[[91, 'Dead End Road', 0, '2024-09-08'],
 [92, 'The Great American Bar Scene', 0, '2024-09-08'],
 [93, 'Boys Back Home', 0, '2024-09-08'],
 [94, 'Purple Gas', 0, '2024-09-08']]

In [21]:
df_song_raw = pd.concat(
    [
        df_song_raw
        , pd.DataFrame(
            np.array(array_new_songs)
            , columns = ['song_id', 'song_name', 'untracked_weeks_on_chart', 'last_update']
        )
    ]
    , ignore_index = True
)

In [37]:
# Update untracked_weeks_on_chart if needed
df_song_raw[-len(array_new_songs)-1:]

,song_id,song_name,untracked_weeks_on_chart,last_update
89,90,Have The Heart,0.0,2024-08-30
90,91,Dead End Road,0,2024-09-08
91,92,The Great American Bar Scene,0,2024-09-08
92,93,Boys Back Home,6,2024-09-08
93,94,Purple Gas,0,2024-09-08


In [36]:
# Adjust untracked weeks if they aren't zero
# Example: df_song_raw.loc[92, 'untracked_weeks_on_chart'] = 6

In [38]:
df_song_raw.to_csv('source_data/song.csv', index=False)

## Add all new songs to the song_to_artist table CSV

In [39]:
df_artist[df_artist['artist_name'].isin(new_song_artists)]

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
4,5,Jelly Roll,0.0,2024-08-23
6,7,Dylan Scott,0.0,2024-08-23
61,62,Dylan Marlowe,0,2024-09-08
62,63,Noeline Hofmann,1,2024-09-08


In [40]:
df_s2a_raw = pd.read_csv('source_data/song_to_artist.csv').fillna('')

In [41]:
array_new_s2a = []
for s2a in [ # song_id, artist_id, relationship_conjunction, relationship_index
    [91, 5, '', '']
    , [92, 1, '', '']
    , [93, 62, '', '']
    , [93, 7, '&', 1]
    , [94, 1, '', '']
    , [94, 63, '&', 1]
]:
    if s2a[3] == '': # Only lead artists don't have a relationship index
        reln = 'Lead artist'
    elif s2a[2] == '': # Featured artists don't have an explicit relationship conjunction
        reln = 'Featured artist' 
    else:
        reln = 'Other main artist'
        
    array_new_s2a += [[
        '-'.join([str(s2a[0]), reln, str(s2a[3])])
        , s2a[0]
        , s2a[1]
        , reln
        , s2a[2]
        , s2a[3]
        , current_date
    ]]
    
array_new_s2a

[['91-Lead artist-', 91, 5, 'Lead artist', '', '', '2024-09-08'],
 ['92-Lead artist-', 92, 1, 'Lead artist', '', '', '2024-09-08'],
 ['93-Lead artist-', 93, 62, 'Lead artist', '', '', '2024-09-08'],
 ['93-Other main artist-1', 93, 7, 'Other main artist', '&', 1, '2024-09-08'],
 ['94-Lead artist-', 94, 1, 'Lead artist', '', '', '2024-09-08'],
 ['94-Other main artist-1', 94, 63, 'Other main artist', '&', 1, '2024-09-08']]

In [42]:
df_s2a_raw = pd.concat(
    [
        df_s2a_raw
        , pd.DataFrame(
            np.array(array_new_s2a)
            , columns = [
                'song_to_artist_pk'
                , 'song_id'
                , 'artist_id'
                , 'relationship_type'
                , 'relationship_conjunction'
                , 'relationship_index'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [43]:
df_s2a_raw.tail(1)

,song_to_artist_pk,song_id,artist_id,relationship_type,relationship_conjunction,relationship_index,last_update
110,94-Other main artist-1,94,63,Other main artist,&,1,2024-09-08


In [44]:
df_s2a_raw.to_csv('source_data/song_to_artist.csv', index=False)

## Convert new_song_to_rank to the id_to_rankix format of existing_song_to_rank
Then stack the two dicts of songs and insert to CSV with more info like current_date for last_update

In [45]:
all_song_to_rank = existing_song_to_rank.copy()
for song_name, rank in new_song_to_rank.items():
    songs = list(df_song_raw[df_song_raw['song_name'] == song_name]['song_id'])
    
    if len(songs) == 1:
        song_id = songs[0]
    elif len(songs) > 1:
        song_id = int(input(f'{str(songs)}: Choose ID of song actually at rank {rank}\n'))
    else:
        raise Exception(f'The song at rank {rank} is missing from the song CSV')
    
    all_song_to_rank[int(song_id)] = rank

In [46]:
array_new_ranking = [
    [
        '-'.join([billboard_observation_date, str(rank)])
        , billboard_observation_date
        , rank
        , song_id
        , current_date
    ]
    for song_id, rank
    in all_song_to_rank.items()
]

array_new_ranking # Make sure it's 50 rows

[['2024-07-31-1', '2024-07-31', 1, 34, '2024-09-08'],
 ['2024-07-31-2', '2024-07-31', 2, 38, '2024-09-08'],
 ['2024-07-31-3', '2024-07-31', 3, 58, '2024-09-08'],
 ['2024-07-31-4', '2024-07-31', 4, 40, '2024-09-08'],
 ['2024-07-31-5', '2024-07-31', 5, 32, '2024-09-08'],
 ['2024-07-31-6', '2024-07-31', 6, 43, '2024-09-08'],
 ['2024-07-31-7', '2024-07-31', 7, 57, '2024-09-08'],
 ['2024-07-31-8', '2024-07-31', 8, 37, '2024-09-08'],
 ['2024-07-31-9', '2024-07-31', 9, 47, '2024-09-08'],
 ['2024-07-31-10', '2024-07-31', 10, 27, '2024-09-08'],
 ['2024-07-31-11', '2024-07-31', 11, 48, '2024-09-08'],
 ['2024-07-31-12', '2024-07-31', 12, 52, '2024-09-08'],
 ['2024-07-31-13', '2024-07-31', 13, 50, '2024-09-08'],
 ['2024-07-31-14', '2024-07-31', 14, 24, '2024-09-08'],
 ['2024-07-31-15', '2024-07-31', 15, 21, '2024-09-08'],
 ['2024-07-31-16', '2024-07-31', 16, 53, '2024-09-08'],
 ['2024-07-31-17', '2024-07-31', 17, 56, '2024-09-08'],
 ['2024-07-31-18', '2024-07-31', 18, 30, '2024-09-08'],
 ['2024-07

In [47]:
df_billboard_ranking_raw = pd.concat(
    [
        pd.read_csv('source_data/billboard_ranking.csv').fillna('')
        , pd.DataFrame(
            np.array(array_new_ranking)
            , columns = [
                'billboard_ranking_pk'
                , 'billboard_observation_date'
                , 'billboard_rank'
                , 'song_id'
                , 'last_update'
            ]
        )
    ]
    , ignore_index = True
)

In [48]:
df_billboard_ranking_raw.to_csv('source_data/billboard_ranking.csv', index=False)

## Push to duckdb

In [49]:
# TODO: instead of purging and rebuilding, use the raw data to run updates
%sql --close duckdb:///C:/Users/Admin/billboard/raw_db.duckdb --quiet
pathlib.Path('raw_db.duckdb').unlink(missing_ok = True)

TODO: integrate into jupyter using `%python`

In a command terminal
- `python table_configs/run_schema_configs.py billboard_schema.sql`
- `python table_configs/run_table_configs.py`
- `python main.py`

## Run the ranking SQL script

In [52]:
with duckdb.connect('raw_db.duckdb') as conn:
    with open('analysis/artist_ranking.sql') as sql_file:
        artist_ranking = conn.sql(sql_file.read())
    
    # TODO: Grab headers from the SQL too
    df_artist_ranking = pd.DataFrame(artist_ranking.fetchall())

In [53]:
df_artist_ranking

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,15,Post Malone,43,43,43,43,43,43,18,18,18,18,18,18,2.0,2.0,2024-07-31
1,1,Zach Bryan,41,41,41,75,75,75,11,11,11,11,11,11,5.0,5.0,2024-07-31
2,2,Morgan Wallen,12,18,24,84,90,96,2,3,4,2,3,4,2.0,2.0,2024-07-31
3,5,Jelly Roll,10,15,23,10,15,29,3,4,6,3,4,6,10.0,10.0,2024-07-31
4,16,Koe Wetzel,12,12,12,12,12,12,2,2,2,2,2,2,9.0,9.0,2024-07-31
5,4,Megan Moroney,11,11,11,11,11,11,3,3,3,3,3,3,25.0,25.0,2024-07-31
6,6,Luke Combs,10,10,17,45,45,52,2,2,4,2,2,4,3.0,3.0,2024-07-31
7,11,Lainey Wilson,10,10,12,10,45,47,2,2,3,2,2,3,19.0,19.0,2024-07-31
8,25,Shaboozey,8,8,8,8,8,8,2,2,2,2,2,2,1.0,1.0,2024-07-31
9,8,Bailey Zimmerman,8,8,8,8,8,8,2,2,2,2,2,2,28.0,28.0,2024-07-31


In [54]:
with duckdb.connect('raw_db.duckdb') as conn:
    print(conn.sql(
        """
        SELECT billboard_observation_date, COUNT(1) n
        FROM raw_db.billboard.billboard_ranking
        GROUP BY 1
        ORDER BY 1 ASC
        """
    ))

┌────────────────────────────┬───────┐
│ billboard_observation_date │   n   │
│            date            │ int64 │
├────────────────────────────┼───────┤
│ 2024-07-31                 │    50 │
│ 2024-08-07                 │    50 │
│ 2024-08-13                 │    50 │
│ 2024-08-23                 │    50 │
│ 2024-08-27                 │    50 │
│ 2024-09-05                 │    50 │
└────────────────────────────┴───────┘

